安装 LLaMA Factory 依赖

In [1]:
%cd /content/
%rm -rf LLaMA-Factory
!git clone --depth 1 https://github.com/hiyouga/LLaMA-Factory.git
%cd LLaMA-Factory
%ls
!pip install torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip uninstall -y jax
!pip install -e .[torch,bitsandbytes,liger-kernel]

/content
Cloning into 'LLaMA-Factory'...
remote: Enumerating objects: 329, done.
remote: Counting objects: 100% (329/329), done.
remote: Compressing objects: 100% (264/264), done.
remote: Total 329 (delta 80), reused 220 (delta 51), pack-reused 0 (from 0)
Receiving objects: 100% (329/329), 8.99 MiB | 15.14 MiB/s, done.
Resolving deltas: 100% (80/80), done.
/content/LLaMA-Factory
assets/       docker/      LICENSE      pyproject.toml  requirements.txt  src/
CITATION.cff  evaluation/  Makefile     README.md       scripts/          tests/
data/         examples/    MANIFEST.in  README_zh.md    setup.py
Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.2.dev0-0.editable-py3-

检查gpu

In [ ]:
import torch
try:
  assert torch.cuda.is_available() is True
except AssertionError:
  print("需要 GPU 环境，申请教程：https://zhuanlan.zhihu.com/p/642542618")

要训练的数据放在data目录下，准备训练数据ry.json


sharegpt格式
```
[
  {
    "conversations": [
      {
        "from": "human",
        "value": "你能解释一下算法分析中的大O记法概念吗？"
      },
      {
        "from": "gpt",
        "value": "是的，大O表示法是一种描述算法性能随输入数据大小增加的方法。它用于估计算法在最坏情况下的时间复杂度或空间复杂度。大O中的“O”代表“数量级”，这意味着随着输入大小的增加，算法所需的操作数以一定的速率增长。\n\n例如，如果我们有一个需要n步才能解决大小为n的问题的算法，那么时间复杂度可以表示为O(n)，这意味着操作数随着输入大小线性增长。如果我们有一个需要n^2步才能解决大小为n的问题的算法，那么时间复杂度可以表示为O(n^2)，这意味着操作数随着输入大小二次方增长。\n\n还有其他几种大O表示法，如O(log n)、O(n log n)、O(2^n)等，它们代表了随着输入大小增加的不同操作数增长速率。通过分析算法的大O表示法，我们可以比较不同算法的性能，并选择给定问题最有效的算法。\n\n您是否有关于大O表示法或算法分析的特定问题？"
      }
    ],
    "system": "",
    "tools": ""
  }
]
```



在dataset_info.json中添加训练数据ry.json



```
"ry": {
    "file_name": "ry.json",
    "formatting": "sharegpt",
    "columns": {
      "messages": "conversations",
      "system": "system",
      "tools": "tools"
    },
    "tags": {
      "role_tag": "from",
      "content_tag": "value",
      "user_tag": "human",
      "assistant_tag": "gpt"
    }
  }
```



基于qwen2.5:0.5b模型进行微调

In [10]:
import json

args = dict(
  stage="sft",                        # 进行指令监督微调
  do_train=True,
  model_name_or_path="Qwen/Qwen2.5-0.5B",
  dataset="ry",      #
  template="qwen",                     # 使用 llama3 提示词模板
  finetuning_type="lora",                   # 使用 LoRA 适配器来节省显存
  lora_target="all",                     # 添加 LoRA 适配器至全部线性层
  output_dir="qwen_lora",                  # 保存 LoRA 适配器的路径
  per_device_train_batch_size=2,               # 批处理大小
  gradient_accumulation_steps=4,               # 梯度累积步数
  lr_scheduler_type="cosine",                 # 使用余弦学习率退火算法
  logging_steps=10,                      # 每 10 步输出一个记录
  warmup_ratio=0.1,                      # 使用预热学习率
  save_steps=1000,                      # 每 1000 步保存一个检查点
  learning_rate=5e-5,                     # 学习率大小
  num_train_epochs=3.0,                    # 训练轮数
  max_samples=300,                      # 使用每个数据集中的 300 条样本
  max_grad_norm=1.0,                     # 将梯度范数裁剪至 1.0
  loraplus_lr_ratio=16.0,                   # 使用 LoRA+ 算法并设置 lambda=16.0
  fp16=True,                         # 使用 float16 混合精度训练
  use_liger_kernel=True,                   # 使用 Liger Kernel 加速训练
)

json.dump(args, open("train_qwen.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

!llamafactory-cli train train_qwen.json

/content/LLaMA-Factory
2025-01-14 07:21:08.980727: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-14 07:21:09.007417: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-14 07:21:09.018723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-14 07:21:09.041546: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 07:21:10.68

将微调模型转成gguf格式

In [3]:
%cd /content/
!git clone https://github.com/ggerganov/llama.cpp.git
!python llama.cpp/convert_lora_to_gguf.py /content/LLaMA-Factory/qwen_lora


/content
Cloning into 'llama.cpp'...
remote: Enumerating objects: 41240, done.
remote: Counting objects: 100% (7557/7557), done.
remote: Compressing objects: 100% (547/547), done.
remote: Total 41240 (delta 7271), reused 7021 (delta 7009), pack-reused 33683 (from 2)
Receiving objects: 100% (41240/41240), 72.68 MiB | 22.05 MiB/s, done.
Resolving deltas: 100% (30059/30059), done.
Traceback (most recent call last):
  File "/content/llama.cpp/convert_lora_to_gguf.py", line 317, in <module>
    lora_model = torch.load(input_model, map_location="cpu", weights_only=True)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 1319, in load
    with _open_file_like(f, "rb") as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 659, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 640, in __init__
    super().__init__(open(name, mode))
FileNot

**创建Modelfile文件**


```
From qwen2.5:0.5b
ADAPTER /content/LLaMA-Factory/qwen_lora/Qwen_Lora-F16-LoRA.gguf
```



安装ollama，本地运行微调之后的模型

In [4]:
!curl -fsSL https://ollama.com/install.sh | sh
!ollama pull qwen2.5:0.5b
!ollama create qwen_lora -f ./Modelfile
!ollama run qwen_lora

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Error: could not connect to ollama app, is it running?
Error: could not connect to ollama app, is it running?
Error: could not connect to ollama app, is it running?


将微调模型压缩成zip包

In [11]:
!zip -r /content/LLaMA-Factory/file.zip /content/LLaMA-Factory/qwen_lora

  adding: content/LLaMA-Factory/qwen_lora/ (stored 0%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/ (stored 0%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/scheduler.pt (deflated 54%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/vocab.json (deflated 61%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/README.md (deflated 66%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/trainer_state.json (deflated 55%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/rng_state.pth (deflated 25%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/tokenizer_config.json (deflated 83%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/added_tokens.json (deflated 67%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/merges.txt (deflated 57%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/special_tokens_map.json (deflated 69%)
  adding: content/LLaMA-Factory/qwen_lora/checkpoint-3/training_args.bin (deflated 51%)
  adding: content

下载zip包

In [12]:
from google.colab import files
files.download("/content/LLaMA-Factory/file.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>